# Bobcat Migration

## Species Description


Bobcats are part of the larger genus of Lynx. There are 4 extant species
of Lynx, one of which is the bobcat. The bobcat is often confused with the
Lynx, but the key difference is that Lynx are larger and their tails are 
all black versus the Bobcat which have white tails with a black strip.

I chose the bobcat *'lynx rufus'* to look at their migration because I 
have a sister who works at the Sonoma County Wildlife Rescue in 
California. She does the [rescue and release](https://scwildliferescue.org/rescue-rehab-release) 
part of the porgram and they frequently rescue injured bobcats, or get in baby 
bobcats that were abandoned in some way that they care for until they 
are old enough to be released. I however, am in Colorado and I have seen
bobcats in the wilderness. This made me curious about their overall migration.

This species's migration ranges between Mexico and Canada, with the 
majority of the bobcats being found in the U.S. They are not a threatened
species, but do suffer from habitat loss. They "live in a wide variety of
habitats, including boreal coniferous and mixed forests in the north, 
bottomland hardwood forests and coastal swamps in the southeast, and 
desert and scrublands in the southwest"(The Smithsonian National Zoo &
Conservation Biology Institute). Another important note that will
inform the later interpretation is that bobcats do not hibernate and are
most active during the winter, particularly January and February which 
is their mating season (The Wildlife Rescue Legaue).

If you are insterested in learning more about bobcats
please visit [The Smithsonian's National Zoo & Conservation Biology 
Institute](https://nationalzoo.si.edu/animals/bobcat) and [The Wildlife
Rescue League](https://www.wildliferescueleague.org/animals/the-bobcat-master-of-the-art-of-concealment/#:~:text=Bobcats%20do%20not%20migrate%2C%20although,large%20as%20six%20square%20miles) .


## Data Description
There were two sources of data used for this project - *Ecoregions* and *GBIF*
data. Ecoregions data is available via a shapefile which can be downloaded
[here](https://www.geographyrealm.com/terrestrial-ecoregions-gis-data/) .
The GBIF data is for species occurances. Together, the two can convey the 
migration spatially and over time of a certain species (if that species' data
is available from GBIF).

* #### Ecoregion Data
Ecoregions "are areas that are geographically and ecologically similar
and experience similar levels of solar radiation and precipitation". 
There are 846 Ecoregions globally, but most species would be in a small
portion of those edoregions depending on their habitats and migration patterns.
The Ecoregion shapefile used is an updated version from 2017 that was developed
by experts.


**Ecoregion Citation:** 

Terauds, Aleks, et al. “Announcing the Release of Ecoregion Snapshots.” 
    >One Earth, One Earth, 31 May 2024, 
    >www.oneearth.org/announcing-the-release-of-ecoregion-snapshots/. 

* #### GBIF Occurences Data

GBIF stands for [Global Biodiversity Information Facility](https://www.gbif.org/what-is-gbif), 
it "is an international network and data infrastructure funded by the world's 
governments and aimed at providing anyone, anywhere, open access to data 
about all types of life on Earth"(Data Info citation below). This network
draws many data sources together including museum specimans, DNA barcodes,
and crowdsourced photos from smartphones used by non-experts and experts.
GBIF uses data standards to index all these species records.

The data can vary:
1. There may be more occurances recorded in National Parks 
compared to the Arctic, even if the species may similarly present
in different regions. This is because there are fewer people to observe in 
the arctic. 
2. There may be greater or fewer occurances depending on time of year 
that people go outside, how accesible a region is during different times
of year, etc.
3. There may be variation depending on how many people want to provide/upload
data and that also depends on what that person knows or likes more - one
may be more likely to upload species they like or know.

Because this data laregly comes from crowdsourcing and the data will need to be 
normalized which will be further explained in the Methods Description.

**GBIF Citation:**

* Data Info:

    https://www.gbif.org/what-is-gbif


* Data:
 
    GBIF.org (12 October 2024) GBIF Occurrence Download
    https://doi.org/10.15468/dl.sye4x3




## Methods Description 

Because the GBIF data can vary by space and time, the data needs to be 
normalized to account for these issues that would otherwise skew the plot
(ex. make it look like there are more observations in a certain area or time
of year, when that wouldn't be an accurate reflection of reality).
The data was normalized by ecoregion samples, by month samples, and by 
area. This way the normalization is over space and time (This should help
control for the number of active observers in each location and time of 
year.)

1. Before normalization can happen, the GBIF data was converted into a 
GeoDataFrame using latitude and longitude geometry.

2. Next, a spatial join was performed with parameters (how= 'inner', 
predicate= 'contains'). This identifies the Ecoregion for each observation.

3. Next, observations were grouped by Ecoregion, selecting only month/ecosystem
combinations that have more than one occurance recorded (since a single
occurence could be an error). The .groupby() and .mean() methods were used
to compute the mean occurences by ecoregion and by month.

4. Lastly, divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion.

This normalizes the data to be able to have a more accurate plot.

## Access locations and times of Bobcat encounters

I used a database called the [Global
Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF
is compiled from species observation data all over the world, and
includes everything from museum specimens to photos taken by citizen
scientists in their backyards.

Before you get started, go to the <a
href="https://www.gbif.org/occurrence/search">GBIF occurrences search
page</a> and explore the data.

### Set up code to prepare for download

I will be getting data from a source called [GBIF (Global Biodiversity
Information Facility)](https://www.gbif.org/). I need a package called
`pygbif` to access the data, which may not be included in my
environment. Install it by running the cell below:

In [ ]:
%%bash
pip install pygbif

### Import Packages

Import packages that will help with
1. reproducible file paths
2. tabular data

In [2]:
import os
import pathlib
import time
import zipfile
from getpass import getpass
from glob import glob

import pandas as pd
import pygbif.occurrences as occ
import pygbif.species as species

In [3]:
# Create data directory in the home folder
data_dir_bobcat = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'species_distribution_bobcat',
)
os.makedirs(data_dir_bobcat, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir_bobcat = os.path.join(data_dir_bobcat, 'gbif_bobcat')

### Check the location for the data_dir_bobcat

In [ ]:
data_dir_bobcat

### Register and log in to GBIF

I need a [GBIF account](https://www.gbif.org/) to complete this
challenge. Then, run the following code to save my credentials on your
computer.

> **Warning**
>
> My email address **must** match the email I used to sign up for
> GBIF!

> **Tip**
>
> If I accidentally enter your credentials wrong, I can set
> `reset_credentials=True` instead of `reset_credentials=False`.

In [5]:
reset_credentials = False
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, 'GBIF username:'),
    GBIF_PWD=(getpass, 'GBIF password'),
    GBIF_EMAIL=(input, 'GBIF email')
)
for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

### Check and make sure you username is correct 

Also double check that the password has been saved

In [ ]:
os.environ ['GBIF_USER']

In [ ]:
!echo $GBIF_USER

In [ ]:
'GBIF_PWD' in os.environ

### Get the species key

1.  Replace the `species_name` with the name of the species you want
 to look up
2.  Run the code to get the species key

In [ ]:
# Query species
species_info = species.name_lookup('lynx rufus', rank='SPECIES')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (nubKey)
species_key = first_result['nubKey']

# Check the result
first_result['species'], species_key

### Download data from GBIF

Submit a request to GBIF

1.  Replace `csv_file_pattern` with a string that will match **any**
    `.csv` file when used in the `glob` function. HINT: the character
    `*` represents any number of any values except the file separator
    (e.g. `/`)

2.  Add parameters to the GBIF download function, `occ.download()` to
    limit your query to:

    -   observations
    -   from 2023
    -   with spatial coordinates.

3.  Then, run the download. **This can take a few minutes**. :::

In [10]:
# Only download once
gbif_pattern = os.path.join(gbif_dir_bobcat, '*.csv')
if not glob(gbif_pattern):
    # Only submit one request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        # Submit query to GBIF
        gbif_query = occ.download([
            "speciesKey = 2435246",
            "hasCoordinate = True",
            "year = 2023",
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']
    wait = occ.download_meta(download_key)['status']
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']
        time.sleep(5)

    # Download GBIF data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=data_dir_bobcat)

    # Unzip GBIF data
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir_bobcat)

# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]

### Check the gbif_path

In [ ]:
gbif_path

### Load the GBIF data into Python

Load GBIF data

1. Look at the beginning of the file I downloaded using the code
below. The delimiter is a tab.
2. Run the following code cell. 
3. Uncomment and modify the parameters of <code>pd.read_csv()</code>
below until my data loads successfully and I have only the columns
I want.

I can use the following code to look at the beginning of my file:

In [ ]:
!head -n 2 $gbif_path 

In [ ]:
# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    #usecols=[]
)
gbif_df.info()

In [ ]:
# Load the GBIF data
bobcat_gbif_df = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    usecols=['gbifID', 'month', 'decimalLatitude', 'decimalLongitude']
)
bobcat_gbif_df.head()

### Import geopandas to use in  next cell

In [15]:
import geopandas as gpd

### Download and save ecoregion boundaries

The ecoregion boundaries take some time to download – they come in at
about 150MB. To use my time most efficiently, I need to complete **caching**
the ecoregions data on the machine I'm working on so that I only
have to download once. To do that, I'll also use **conditionals**, or 
code that adjusts what it does based on the situation.

Get ecoregions boundaries
1. Find the URL for for the ecoregion boundary
<strong>Shapefile</strong>. I can <a
href="https://www.geographyrealm.com/terrestrial-ecoregions-gis-data/">get
ecoregion boundaries from Google.</a>.
2. Change all the variable names to <strong>descriptive</strong>
variable names, making sure to correctly reference variables I created
before.
3. Run the cell to download and save the data.

In [16]:
# Set up the ecoregion boundary URL
ecoregions_url = (
 "https://storage.googleapis.com/teow2016"
 "/Ecoregions2017.zip")

# Set up a path to save the data on your machine
ecoregions_dir = os.path.join(data_dir_bobcat, 'resolve_ecoregions')

# Make the ecoregions directory
os.makedirs(ecoregions_dir, exist_ok=True)

# Join ecoregions shapefile path
ecoregions_path = os.path.join(ecoregions_dir, 'ecoregions.shp')

# Only download once
if not os.path.exists(ecoregions_path):
    ecoregions_gdf = gpd.read_file(ecoregions_url)
    ecoregions_gdf.to_file(ecoregions_path)

Make sure that worked! Use a **bash** command
called `find` to look for all the files in my project directory with
the `.shp` extension:

In [ ]:
%%bash
find ~/earth-analytics/data/species_distribution_bobcat -name '*.shp' 

### Load Ecoregions into Python

Download and save ecoregion boundaries from the EPA:
1. Make a quick plot with <code>.plot()</code> to make sure the
download worked.
2. Run the cell to load the data into Python

In [ ]:
# Open up the ecoregions boundaries
ecoregions_gdf = gpd.read_file(ecoregions_path)

# Name the index so it will match the other data later on
ecoregions_gdf.index.name = 'ecoregion'

# Plot the ecoregions to check download
ecoregions_gdf.plot(edgecolor='black', color='lightgreen')

In [ ]:
ecoregions_gdf.head()

### Convert the GBIF data to a GeoDataFrame

To plot the GBIF data, I need to convert it to a `GeoDataFrame` first.
This will make some special geospatial operations from `geopandas`
available, such as spatial joins and plotting.

1. Convert `DataFrame` to `GeoDataFrame`
2. Run the code to get a <code>GeoDataFrame</code> of the GBIF
data.

In [ ]:
#Convert dataframe (df) into geo dataframe (gdf)
bobcat_gbif_gdf = (
    gpd.GeoDataFrame(
        bobcat_gbif_df, 
        geometry=gpd.points_from_xy(
            bobcat_gbif_df.decimalLongitude, 
            bobcat_gbif_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month', 'geometry']]
)
bobcat_gbif_gdf

### Store the new version of your dataframe for other notebooks as needed

In [ ]:
%store ecoregions_gdf bobcat_gbif_gdf

## Normalize Data



### Identify the Ecoregion for Each Observation

Combine the ecoregions and the observations **spatially** using
a method called `.sjoin()`, which stands for spatial join.

1. Perform a spatial join
2. Identify the correct values for the <code>how=</code> and
<code>predicate=</code> parameters of the spatial join.
3. Select only the columns I will need for my plot.
4. Run the code.

In [ ]:
gbif_ecoregion_gdf = (
    ecoregions_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(bobcat_gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        bobcat_gbif_gdf,
        how='inner', 
        predicate='contains')
    # Select the required columns
    [['month','ECO_NAME','gbifID', 'OBJECTID']]

    # rename columns as needed
    .reset_index()
    .rename(columns={
       'ECO_NAME': 'name',
       'gbifID': 'observation_id',
       'OBJECTID': 'object_id'})
)

gbif_ecoregion_gdf

### Count the Observations in Each Ecroregion Each Month

Group observations by ecoregion

1. Select only month/ecosystem combinations that have more than one
occurrence recorded, since a single occurrence could be an error.
2. Use the <code>.groupby()</code> and <code>.mean()</code> methods to
compute the mean occurrences by ecoregion and by month.
3. Run the code – it will normalize the number of occurrences by month
and ecoregion

In [ ]:
bobcat_occurrence_df = (
    gbif_ecoregion_gdf
    # For each ecoregion, for each month...
    .groupby(['ecoregion', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('observation_id', 'count'))
)

# Get rid of rare observations (possible misidentification?)
bobcat_occurrence_df = bobcat_occurrence_df[bobcat_occurrence_df.occurrences>1]

bobcat_occurrence_df

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
     bobcat_occurrence_df
     .groupby(['ecoregion'])
     .mean()
 )
# Take the mean by month
mean_occurrences_by_month = (
     bobcat_occurrence_df
     .groupby(['month'])
     .mean()
 )

bobcat_occurrence_df

In [ ]:
mean_occurrences_by_ecoregion

In [ ]:
mean_occurrences_by_month

### Normalize the Observations

Divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion

In [ ]:
# Normalize by space and time for sampling effort
bobcat_occurrence_df['norm_occurrences'] = (
    bobcat_occurrence_df 
    /mean_occurrences_by_month 
    /mean_occurrences_by_ecoregion
)
bobcat_occurrence_df

### Store the new version of your dataframe for other notebooks as needed

In [ ]:
%store bobcat_occurrence_df

## Plot Data

### Import Packages Needed

Import packages for making interactive maps with vector data as well as
calendar in order to get month names that will be used when creating the]plot.

In [ ]:
# Get month names
import calendar

# Libraries for Dynamic mapping
import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import geopandas as gpd
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv
import hvplot.pandas
import panel as pn

### Create a Simplified GeoDataFrame for Plotting

The code below will
streamline plotting with `hvplot` by simplifying the geometry,
projecting it to a Mercator projection that is compatible with
`geoviews`, and cropping off areas in the Arctic.

Download and save ecoregion boundaries from the EPA:</p>
<ol type="1">
<li>Simplify the ecoregions with <code>.simplify(.05)</code>, and save
it back to the <code>geometry</code> column.</li>
<li>Change the Coordinate Reference System (CRS) to Mercator with
<code>.to_crs(ccrs.Mercator())</code></li>
<li>Use the plotting code that is already in the cell to check that the
plotting runs quickly (less than a minute) and looks the way you want.</li>
</ol></div></div>

In [ ]:
# Simplify the geometry to speed up processing
ecoregions_gdf.geometry = ecoregions_gdf.simplify(
    .01, preserve_topology=False
    )

# Change the CRS to Mercator for mapping
ecoregions_gdf = ecoregions_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
ecoregions_gdf.hvplot(
    x='Longitude',
    y='Latitude',
    geo=True, 
    crs=ccrs.Mercator()
    )

### Map Migration Over Time


1. If applicable, replace any variable names with the names I defined
previously.
2. Customize my plot with my choice of title, tile source, color
map, and size.

<div data-__quarto_custom="true" data-__quarto_custom_type="Callout"
data-__quarto_custom_context="Block" data-__quarto_custom_id="3">
<div data-__quarto_custom_scaffold="true">

</div>
<div data-__quarto_custom_scaffold="true">
<p>My plot will probably still change months very slowly in my
Jupyter notebook, because it calculates each month’s plot as needed.
Open up the saved HTML file to see faster performance!</p>
</div>


In [ ]:
# Join the occurrences with the plotting GeoDataFrame
bobcat_occurrence_gdf = ecoregions_gdf.join(bobcat_occurrence_df)

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = bobcat_occurrence_gdf.total_bounds

# Define the slider widget
slider = pn.widgets.DiscreteSlider(
    name='month', 
    options={calendar.month_name[i]: i for i in range(1, 13)}
)

# Plot occurrence by ecoregion and month
bobcat_migration_plot = (
    bobcat_occurrence_gdf
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Bobcat Migration Observations Over Time",
        x='Longitude',
        y='Latitude',
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        width=600,
        height=600,
        widgets={'month': slider},
        widget_location='bottom'
    )
)

# Save the plot
bobcat_migration_plot.save('bobcat_migration.html', embed=True)

# Show the plot
bobcat_migration_plot

## Bobcat Migration Overtime Plot- While this species does move around, they are not migratory. The map reflects the ecoregions they habitate as well as the time of year they would be more active. 


<embed type="text/html" src="bobcat_migration.html",
width="900" height="900">


The Bobcat Migration Overtime shows a predominance in the U.S. thoughout
most of the year and fewer occurences in Canada and Mexico. There is not 
a striaght forward migration pattern that may be seen in other species
that may 'fly south for the winter' leading to what I have now read in multiple
places is that - bobcats are not migratory. This was not something I researched
prior to delving into the data, but after seeing the plot makes sense. 
The plot also confirms some of the other species information.

June through August tends to have fewer norm occurences in general compared to
the fall and winter months (for the Northern Hemisphere) which have higher
norm occurences. This speaks to the fact that bobcats don't hibernate and 
winter is their mating season, so they would be more likely to be spotted
by a human while looking for a mate. Bobcats are also found around areas
often frequented for skiing/snowboarding and snowshoeing. 

The lack of a straight forward migration pattern speaks to the nature of
bobcats, like the much smaller domestic housecat, are territorial animals.
They tend to occupy the same milage over time, marking it, and only move
under extreme conditions or habitat loss. Because they live in different
types of forests, coastal swamps, and scrubland, they would be threatend
by wildfire and drought. There have been an increase in wildfires in recent
years in the Sierra Nevadas, the Rocky Mountains, and possibly other forest
areas which would threaten or cause them to relocate.


The plot more so speaks to the nature of bobcats and how the data was collected
than it speaks to a distinct migration pattern.

Discussion and conclusion ideally link back to the earlier text for context
(the species description, data, and methods stuff from up top) and interpret
the plot as well